<a href="https://colab.research.google.com/github/arssite/Datalysis/blob/main/4Model_testing_for_PCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence-transformers transformers adapters torch

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from adapters import AutoAdapterModel
import time

AttributeError: partially initialized module 'torch' has no attribute 'types' (most likely due to a circular import)

In [ ]:
import json
from google.colab import files

In [ ]:
# === 2. Define Helper Pooling Functions ===

# For SciBERT (and bge-m3 if we used AutoModel)
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

# For Specter-2
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

# === 3. Define Device ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Running on {device} ---")

--- Running on cpu ---


In [ ]:
# === 1. Upload your JSON file ===
print("Please upload your 'chunks (1).json' file...")
uploaded = files.upload()

# Get the name of the uploaded file
file_name = list(uploaded.keys())[0]
print(f"Uploaded file '{file_name}'")

# === 2. Load and Parse the Data ===
real_test_chunks = []
with open(file_name, 'r') as f:
    data = json.load(f)
    for i, item in enumerate(data):
        # We store the content AND its original chunk number for reference
        real_test_chunks.append({
            "id": i,
            "content": item["page_content"]
        })

print(f"Successfully loaded {len(real_test_chunks)} chunks from your file.")

Please upload your 'chunks (1).json' file...


Saving test_chunks.json to test_chunks (5).json
Uploaded file 'test_chunks (5).json'
Successfully loaded 14 chunks from your file.


In [ ]:
# === 1. Define Queries for YOUR Data ===
# These are your new, "hardest" queries.
real_test_queries = [
    ("Q1_ion_formation", "Who is Elon Musk?"),
    ("Q2_dissociation", "Show me the equation for one mole of ionic compound dissociating into gaseous ions"),
    ("Q3_chlorine_dissociation", "What is the equation for half a mole of chlorine gas producing chlorine atoms?"),
    ("Q4_enthalpy_calculation", "What is the dissociation enthalpy for producing one mole of chlorine atoms from 242 kJ?"),
    ("Q5_temperature_conversion", "What is the formula to convert Celsius to Kelvin?"),
    ("Q6_body_temperature", "What is human body temperature in Kelvin and Celsius?"),
    ("Q7_density_units", "What are the SI units for density expressed as mass per volume?"),
    ("Q8_acceleration_units", "How do you derive the SI units for acceleration from velocity and time?"),
    ("Q9_pressure_units", "What are the derived SI units for pressure from force and area?"),
    ("Q10_force_units", "What is the SI unit for force derived from mass and acceleration?"),
    ("Q11_sublimation_enthalpy", "What is the sublimation enthalpy of sodium in kilojoules per mole?"),
    ("Q12_ionization_enthalpy", "What is the ionization enthalpy for converting gaseous sodium atoms to sodium ions?"),
    ("Q13_electron_gain_enthalpy", "What is the electron gain enthalpy of chlorine atoms to chloride ions?"),
    ("Q14_formation_enthalpy", "What is the enthalpy of formation for sodium chloride from sodium metal and chlorine gas?"),
    ("Q15_electron_loss", "Show the equation where sodium atom loses an electron to become an ion"),
    ("Q16_electron_gain", "What is the equation for chlorine atom gaining an electron?"),
    ("Q17_volume_calculation", "If a box has dimensions 1.0 m, 2.0 m, and 3.0 m, what is its volume in cubic meters?"),
    ("Q18_area_derivation", "How do you derive square meters as the unit of area?"),
    ("Q19_delta_symbol", "What symbol is used to represent enthalpy change in thermodynamic processes?"),
    ("Q20_acceleration_derivation", "Show the mathematical derivation of acceleration units from meters per second"),
    ("Q21_born_haber_cycle", "List all the steps with their enthalpy changes to form NaCl from solid sodium and gaseous chlorine"),
    ("Q22_unit_conversion_chain", "Show how to derive kg m^{-1} s^{-2} from force divided by area"),
    ("Q23_negative_enthalpy", "Which enthalpy changes are negative in the Born-Haber cycle for NaCl?"),
    ("Q24_nacl_dissociation_latex", "Show me the equation: \\mathrm{NaCl}(s) \\longrightarrow \\mathrm{Na}^{+}(g)+\\mathrm{Cl}^{-}(g)"),
    ("Q25_sublimation_latex", "What is \\mathrm{Na}(s) \\longrightarrow \\mathrm{Na}(\\mathrm{g}) enthalpy value?"),
    ("Q26_chlorine_half_mole_latex", "Find the equation: \\frac{1}{2} \\mathrm{Cl}_{2}(g) \\longrightarrow \\mathrm{Cl}(g)"),
    ("Q27_ionization_latex", "What is the enthalpy for \\mathrm{Na}(g) \\longrightarrow \\mathrm{Na}^{+}(g)+e^{-}(g)?"),
    ("Q28_electron_gain_latex", "Find enthalpy for \\mathrm{Cl}(g)+e^{-} \\longrightarrow \\mathrm{Cl}^{-}(g)"),
    ("Q29_lattice_enthalpy_latex", "Show equation: \\mathrm{Na}^{+}(g)+\\mathrm{Cl}^{-}(g) \\longrightarrow \\mathrm{NaCl}(s)"),
    ("Q30_formation_latex", "What is enthalpy for \\mathrm{Na}(s)+\\frac{1}{2} \\mathrm{Cl}_{2}(g) \\longrightarrow \\mathrm{NaCl}(s)?"),
    ("Q31_density_formula_latex", "Find the formula: \\text{Density} = \\frac{\\text{Mass}}{\\text{Volume}} = \\mathbf{kg} \\mathbf{m}^{-3}"),
    ("Q32_acceleration_latex", "Show derivation: \\text{Acceleration} = \\frac{\\text{Velocity}}{\\text{Time}} = \\mathbf{m} \\mathbf{s}^{-2}"),
    ("Q33_pressure_units_latex", "Find units: \\text{Pressure} = \\frac{\\mathrm{kg} \\mathrm{~m} \\mathrm{~s}^{-2}}{\\mathrm{~m}^{2}} = \\mathbf{kg} \\mathbf{m}^{-1} \\mathbf{s}^{-2}"),
    ("Q34_kelvin_conversion_latex", "What is the formula: \\mathrm{K} = {^{\\circ} \\mathrm{C}} + 273.15?"),
    ("Q35_delta_sub_h", "Find the value of \\Delta_{\\text{sub}} \\mathrm{H}^{\\circ} for sodium"),
    ("Q36_delta_i_h", "What is \\Delta_{\\mathrm{i}} \\mathrm{H}^{\\circ} for sodium ionization?"),
    ("Q37_delta_eg_h", "Find \\Delta_{eg} \\mathrm{H}^{\\circ} for chlorine electron gain"),
    ("Q38_delta_f_h", "What is \\Delta_{\\mathrm{f}} \\mathrm{H}^{\\circ} for NaCl formation?"),
    ("Q39_volume_formula_latex", "Show: \\text{Volume} = \\mathrm{m} \\times \\mathrm{m} \\times \\mathrm{m} = \\mathrm{m}^{3}"),
    ("Q40_force_formula_latex", "Find: \\text{Force} = \\text{Mass} \\times \\text{Acceleration} = \\mathbf{kg} \\mathbf{m} \\mathbf{s}^{-2}")
]

print(f"Defined {len(real_test_queries)} new test queries.")

Defined 40 new test queries.


In [ ]:
print("\n" + "="*80)
print("🚀 TESTING MODEL: BAAI/bge-m3")
print("="*80)

# --- 1. Load Model ---
model_name = "BAAI/bge-m3"
model = SentenceTransformer(model_name, device=device)
print(f"Loaded {model_name}.")

# --- 2. Create Chunk Embeddings (The "Database") ---
chunk_content_list = [chunk['content'] for chunk in real_test_chunks]
with torch.no_grad():
    chunk_embeddings = model.encode(chunk_content_list, normalize_embeddings=True)
    chunk_embeddings = torch.tensor(chunk_embeddings).to(device)

# --- 3. Run All Queries ---
for q_name, q_text in real_test_queries:
    print(f"\n  QUERY: '{q_name}'")

    with torch.no_grad():
        query_vector = model.encode(q_text, normalize_embeddings=True)
        query_vector = torch.tensor(query_vector).to(device)

    # 4. Calculate Scores
    scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)
    top_scores, top_indices = torch.topk(scores, k=3)

    # 5. Print results
    for i in range(3):
        score = top_scores[0][i].item()
        chunk_index = top_indices[0][i].item()
        retrieved_content = real_test_chunks[chunk_index]['content']
        print(f"    {i+1}. [Chunk {chunk_index}] (Score: {score:.4f}) > {retrieved_content[:80]}...")


🚀 TESTING MODEL: BAAI/bge-m3
Loaded BAAI/bge-m3.

  QUERY: 'Q1_ion_formation'
    1. [Chunk 6] (Score: 0.3049) > $$  \nThe definitions of the SI base units are given in Table 2.  \nTable 2. Def...
    2. [Chunk 9] (Score: 0.2843) > ![img-0.jpeg](img-0.jpeg)\n### Figure img-0.jpeg: Types of Thermometers  \nThe i...
    3. [Chunk 5] (Score: 0.2761) > (v) Combination of oppositely charged ions to form a solid crystal. The opposite...

  QUERY: 'Q2_dissociation'
    1. [Chunk 3] (Score: 0.5704) > (iii) Conversion of gaseous sodium atoms into sodium ions. The amount of enthalp...
    2. [Chunk 4] (Score: 0.5346) > (iv) Conversion of gaseous chlorine atoms into gaseous chloride ions. The amount...
    3. [Chunk 2] (Score: 0.5344) > ii) Dissociation of chlorine molecules into chlorine atoms. Each chlorine molecu...

  QUERY: 'Q3_chlorine_dissociation'
    1. [Chunk 2] (Score: 0.6670) > ii) Dissociation of chlorine molecules into chlorine atoms. Each chlorine molecu...
    2. [Chunk 4] (Score

In [ ]:
print("\n" + "="*80)
print("🚀 TESTING MODEL: allenai/scibert_scivocab_uncased")
print("="*80)

# --- 1. Load Model ---
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
print(f"Loaded {model_name}.")

# --- 2. Create Chunk Embeddings (The "Database") ---
chunk_content_list = [chunk['content'] for chunk in real_test_chunks]
with torch.no_grad():
    encoded_input = tokenizer(chunk_content_list, padding=True, truncation=True, return_tensors='pt').to(device)
    model_output = model(**encoded_input)
    chunk_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    chunk_embeddings = F.normalize(chunk_embeddings, p=2, dim=1)

# --- 3. Run All Queries ---
for q_name, q_text in real_test_queries:
    print(f"\n  QUERY: '{q_name}'")

    with torch.no_grad():
        encoded_input = tokenizer(q_text, padding=True, truncation=True, return_tensors='pt').to(device)
        model_output = model(**encoded_input)
        query_vector = mean_pooling(model_output, encoded_input['attention_mask'])
        query_vector = F.normalize(query_vector, p=2, dim=1)

    # 4. Calculate Scores (with the bug fix)
    if query_vector.dim() == 1:
        scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)
    else:
        scores = torch.mm(query_vector, chunk_embeddings.T)

    top_scores, top_indices = torch.topk(scores, k=3)

    # 5. Print results
    for i in range(3):
        score = top_scores[0][i].item()
        chunk_index = top_indices[0][i].item()
        retrieved_content = real_test_chunks[chunk_index]['content']
        print(f"    {i+1}. [Chunk {chunk_index}] (Score: {score:.4f}) > {retrieved_content[:80]}...")


🚀 TESTING MODEL: allenai/scibert_scivocab_uncased


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Loaded allenai/scibert_scivocab_uncased.

  QUERY: 'Q1_ion_formation'
    1. [Chunk 9] (Score: 0.5982) > ![img-0.jpeg](img-0.jpeg)\n### Figure img-0.jpeg: Types of Thermometers  \nThe i...
    2. [Chunk 1] (Score: 0.5870) > The overall process is thought to take place in the following steps :(i) Convers...
    3. [Chunk 4] (Score: 0.5606) > (iv) Conversion of gaseous chlorine atoms into gaseous chloride ions. The amount...

  QUERY: 'Q2_dissociation'
    1. [Chunk 1] (Score: 0.7744) > The overall process is thought to take place in the following steps :(i) Convers...
    2. [Chunk 9] (Score: 0.7451) > ![img-0.jpeg](img-0.jpeg)\n### Figure img-0.jpeg: Types of Thermometers  \nThe i...
    3. [Chunk 4] (Score: 0.7271) > (iv) Conversion of gaseous chlorine atoms into gaseous chloride ions. The amount...

  QUERY: 'Q3_chlorine_dissociation'
    1. [Chunk 9] (Score: 0.7698) > ![img-0.jpeg](img-0.jpeg)\n### Figure img-0.jpeg: Types of Thermometers  \nThe i...
    2. [Chunk 1] (Score: 0.7574)

In [ ]:
print("\n" + "="*80)
print("🚀 TESTING MODEL: allenai/specter2")
print("="*80)

# --- 1. Load Model ---
model_name = "allenai/specter2_base"
adapter_name = "allenai/specter2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoAdapterModel.from_pretrained(model_name).to(device)
model.load_adapter(adapter_name, source="hf", set_active=True)
print(f"Loaded {model_name} with {adapter_name} adapter.")

# --- 2. Create Chunk Embeddings (The "Database") ---
chunk_content_list = [chunk['content'] for chunk in real_test_chunks]
with torch.no_grad():
    encoded_input = tokenizer(chunk_content_list, padding=True, truncation=True, return_tensors='pt').to(device)
    model_output = model(**encoded_input)
    chunk_embeddings = cls_pooling(model_output)
    chunk_embeddings = F.normalize(chunk_embeddings, p=2, dim=1)

# --- 3. Run All Queries ---
for q_name, q_text in real_test_queries:
    print(f"\n  QUERY: '{q_name}'")

    with torch.no_grad():
        encoded_input = tokenizer(q_text, padding=True, truncation=True, return_tensors='pt').to(device)
        model_output = model(**encoded_input)
        query_vector = cls_pooling(model_output)
        query_vector = F.normalize(query_vector, p=2, dim=1)

    # 4. Calculate Scores (with the bug fix)
    if query_vector.dim() == 1:
        scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)
    else:
        scores = torch.mm(query_vector, chunk_embeddings.T)

    top_scores, top_indices = torch.topk(scores, k=3)

    # 5. Print results
    for i in range(3):
        score = top_scores[0][i].item()
        chunk_index = top_indices[0][i].item()
        retrieved_content = real_test_chunks[chunk_index]['content']
        print(f"    {i+1}. [Chunk {chunk_index}] (Score: {score:.4f}) > {retrieved_content[:80]}...")


🚀 TESTING MODEL: allenai/specter2


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Loaded allenai/specter2_base with allenai/specter2 adapter.

  QUERY: 'Q1_ion_formation'
    1. [Chunk 13] (Score: 0.9048) > Elon Reeve Musk is a businessman and entrepreneur known for his leadership of Te...
    2. [Chunk 12] (Score: 0.8751) > \\text { Velocity } & =\\frac{\\text { Distance }}{\\text { Time }}=\\frac{\\mat...
    3. [Chunk 8] (Score: 0.8575) > $$\n\\mathrm{K}={ }^{\\circ} \\mathrm{C}+273.15\n$$  \nor $\\quad \\mathrm{K}={ ...

  QUERY: 'Q2_dissociation'
    1. [Chunk 7] (Score: 0.9109) > The overall energy change may be expressed as :$\mathrm{Na}(s)+\frac{1}{2} \math...
    2. [Chunk 1] (Score: 0.9084) > The overall process is thought to take place in the following steps :(i) Convers...
    3. [Chunk 0] (Score: 0.9073) > The lattice enthalpy of a compound can be calculated from other thermodynamic pr...

  QUERY: 'Q3_chlorine_dissociation'
    1. [Chunk 2] (Score: 0.9357) > ii) Dissociation of chlorine molecules into chlorine atoms. Each chlorine molecu...
    2. [Chu

In [ ]:
print("\n" + "="*80)
print("🚀 TESTING MODEL: ai4stem-uga/G-SciEdBERT")
print("="*80)

# --- 1. Load Model ---
model_name = "ai4stem-uga/G-SciEdBERT"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    print(f"Loaded {model_name}.")
except Exception as e:
    print(f"Failed to load {model_name}: {e}")
    # If model fails to load, we skip the rest of the test
    pass

if 'model' in locals():
    # --- 2. Create Chunk Embeddings (The "Database") ---
    chunk_content_list = [chunk['content'] for chunk in real_test_chunks]
    with torch.no_grad():
        # We must truncate at 512 for BERT-based models
        encoded_input = tokenizer(chunk_content_list, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
        model_output = model(**encoded_input)
        chunk_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        chunk_embeddings = F.normalize(chunk_embeddings, p=2, dim=1)

    # --- 3. Run All Queries ---
    for q_name, q_text in real_test_queries:
        print(f"\n  QUERY: '{q_name}'")

        with torch.no_grad():
            # We must truncate at 512 for BERT-based models
            encoded_input = tokenizer(q_text, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
            model_output = model(**encoded_input)
            query_vector = mean_pooling(model_output, encoded_input['attention_mask'])
            query_vector = F.normalize(query_vector, p=2, dim=1)

        # 4. Calculate Scores (with the bug fix)
        if query_vector.dim() == 1:
            scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)
        else:
            scores = torch.mm(query_vector, chunk_embeddings.T)

        top_scores, top_indices = torch.topk(scores, k=3)

        # 5. Print results
        for i in range(3):
            score = top_scores[0][i].item()
            chunk_index = top_indices[0][i].item()
            retrieved_content = real_test_chunks[chunk_index]['content']
            print(f"    {i+1}. [Chunk {chunk_index}] (Score: {score:.4f}) > {retrieved_content[:80]}...")


🚀 TESTING MODEL: ai4stem-uga/G-SciEdBERT


Some weights of BertModel were not initialized from the model checkpoint at ai4stem-uga/G-SciEdBERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded ai4stem-uga/G-SciEdBERT.

  QUERY: 'Q1_ion_formation'
    1. [Chunk 1] (Score: 0.6822) > The overall process is thought to take place in the following steps :(i) Convers...
    2. [Chunk 4] (Score: 0.6519) > (iv) Conversion of gaseous chlorine atoms into gaseous chloride ions. The amount...
    3. [Chunk 7] (Score: 0.6498) > The overall energy change may be expressed as :$\mathrm{Na}(s)+\frac{1}{2} \math...

  QUERY: 'Q2_dissociation'
    1. [Chunk 6] (Score: 0.8428) > $$  \nThe definitions of the SI base units are given in Table 2.  \nTable 2. Def...
    2. [Chunk 1] (Score: 0.8300) > The overall process is thought to take place in the following steps :(i) Convers...
    3. [Chunk 9] (Score: 0.8195) > ![img-0.jpeg](img-0.jpeg)\n### Figure img-0.jpeg: Types of Thermometers  \nThe i...

  QUERY: 'Q3_chlorine_dissociation'
    1. [Chunk 6] (Score: 0.8408) > $$  \nThe definitions of the SI base units are given in Table 2.  \nTable 2. Def...
    2. [Chunk 1] (Score: 0.8375) > The ov

In [ ]:
print("\n" + "="*80)
print("🚀 TESTING MODEL: intfloat/e5-mistral-7b-instruct")
print("="*80)

# --- 1. Load Model ---
model_name = "intfloat/e5-mistral-7b-instruct"
model = SentenceTransformer(model_name, device=device)
print(f"Loaded {model_name}.")

# --- 2. Create Chunk Embeddings (The "Database") ---
# E5 needs a "passage: " prefix for chunks
passage_chunks = [f"passage: {chunk['content']}" for chunk in real_test_chunks]
with torch.no_grad():
    chunk_embeddings = model.encode(passage_chunks, normalize_embeddings=True)
    chunk_embeddings = torch.tensor(chunk_embeddings).to(device)

# --- 3. Run All Queries ---
for q_name, q_text in real_test_queries:
    print(f"\n  QUERY: '{q_name}'")

    # E5 needs a "query: " prefix for queries
    query_with_prefix = f"query: {q_text}"

    with torch.no_grad():
        query_vector = model.encode(query_with_prefix, normalize_embeddings=True)
        query_vector = torch.tensor(query_vector).to(device)

    # 4. Calculate Scores
    scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)
    top_scores, top_indices = torch.topk(scores, k=3)

    # 5. Print results
    for i in range(3):
        score = top_scores[0][i].item()
        chunk_index = top_indices[0][i].item()
        retrieved_content = real_test_chunks[chunk_index]['content']
        print(f"    {i+1}. [Chunk {chunk_index}] (Score: {score:.4f}) > {retrieved_content[:80]}...")

In [ ]:
print("\n" + "="*80)
print("🚀 TESTING MODEL: jina-ai/jina-embeddings-v2-base-en")
print("="*80)

# --- 1. Load Model ---
model_name = "jina-ai/jina-embeddings-v2-base-en"
model = SentenceTransformer(model_name, device=device, trust_remote_code=True)
print(f"Loaded {model_name}.")

# --- 2. Create Chunk Embeddings (The "Database") ---
chunk_content_list = [chunk['content'] for chunk in real_test_chunks]
with torch.no_grad():
    chunk_embeddings = model.encode(chunk_content_list, normalize_embeddings=True)
    chunk_embeddings = torch.tensor(chunk_embeddings).to(device)

# --- 3. Run All Queries ---
for q_name, q_text in real_test_queries:
    print(f"\n  QUERY: '{q_name}'")

    with torch.no_grad():
        query_vector = model.encode(q_text, normalize_embeddings=True)
        query_vector = torch.tensor(query_vector).to(device)

    # 4. Calculate Scores
    scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)
    top_scores, top_indices = torch.topk(scores, k=3)

    # 5. Print results
    for i in range(3):
        score = top_scores[0][i].item()
        chunk_index = top_indices[0][i].item()
        retrieved_content = real_test_chunks[chunk_index]['content']
        print(f"    {i+1}. [Chunk {chunk_index}] (Score: {score:.4f}) > {retrieved_content[:80]}...")

In [ ]:
print("\n" + "="*80)
print("🚀 TESTING MODEL: jina-ai/jina-embeddings-v2-base-en")
print("="*80)

# --- 1. Load Model ---
model_name = "jina-ai/jina-embeddings-v2-base-en"
model = SentenceTransformer(model_name, device=device, trust_remote_code=True)
print(f"Loaded {model_name}.")

# --- 2. Create Chunk Embeddings (The "Database") ---
with torch.no_grad():
    chunk_embeddings = model.encode(hard_test_chunks, normalize_embeddings=True)
    chunk_embeddings = torch.tensor(chunk_embeddings).to(device)

# --- 3. Run All Queries ---
for q_name, q_text in test_queries:
    print(f"\n  QUERY: '{q_name}'")

    with torch.no_grad():
        query_vector = model.encode(q_text, normalize_embeddings=True)
        query_vector = torch.tensor(query_vector).to(device)

    # 4. Calculate Scores
    scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)
    top_scores, top_indices = torch.topk(scores, k=3)

    # 5. Print results
    for i in range(3):
        score = top_scores[0][i].item()
        chunk_index = top_indices[0][i].item()
        print(f"    {i+1}. [Chunk {chunk_index}] (Score: {score:.4f})")


🚀 TESTING MODEL: jina-ai/jina-embeddings-v2-base-en


OSError: jina-ai/jina-embeddings-v2-base-en is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:
print("\n" + "="*80)
print("🚀 TESTING MODEL: intfloat/e5-mistral-7b-instruct")
print("="*80)

# --- 1. Load Model ---
model_name = "intfloat/e5-mistral-7b-instruct"
model = SentenceTransformer(model_name, device=device)
print(f"Loaded {model_name}.")

# --- 2. Create Chunk Embeddings (The "Database") ---
# E5 needs a "passage: " prefix for chunks
passage_chunks = [f"passage: {chunk}" for chunk in hard_test_chunks]
with torch.no_grad():
    chunk_embeddings = model.encode(passage_chunks, normalize_embeddings=True)
    chunk_embeddings = torch.tensor(chunk_embeddings).to(device)

# --- 3. Run All Queries ---
for q_name, q_text in test_queries:
    print(f"\n  QUERY: '{q_name}'")

    # E5 needs a "query: " prefix for queries
    query_with_prefix = f"query: {q_text}"

    with torch.no_grad():
        query_vector = model.encode(query_with_prefix, normalize_embeddings=True)
        query_vector = torch.tensor(query_vector).to(device)

    # 4. Calculate Scores
    scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)
    top_scores, top_indices = torch.topk(scores, k=3)

    # 5. Print results
    for i in range(3):
        score = top_scores[0][i].item()
        chunk_index = top_indices[0][i].item()
        print(f"    {i+1}. [Chunk {chunk_index}] (Score: {score:.4f})")


🚀 TESTING MODEL: intfloat/e5-mistral-7b-instruct


ImportError: cannot import name 'LossKwargs' from 'transformers.utils' (/usr/local/lib/python3.12/dist-packages/transformers/utils/__init__.py)

In [ ]:
# === 4. Main Test Loop ===
models_to_test = [
    "BAAI/bge-m3",
    "intfloat/e5-mistral-7b-instruct",
    "allenai/specter2_base", # This will also load the adapter
    "allenai/scibert_scivocab_uncased",
    "jina-ai/jina-embeddings-v2-base-en"
]

In [ ]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Running on {device} ---")

--- Running on cpu ---


In [ ]:
for model_id in models_to_test:
    print("\n" + "="*80)
    print(f"🚀 TESTING MODEL: {model_id}")
    print("="*80)

    model = None
    tokenizer = None
    model_type = ""

    # --- Model Loading Logic ---
    start_time = time.time()
    try:
        if "bge-m3" in model_id or "jina" in model_id:
            model_type = "SentenceTransformer"
            # Jina needs trust_remote_code
            trust = "jina" in model_id
            model = SentenceTransformer(model_id, device=device, trust_remote_code=trust)

        elif "e5-mistral" in model_id:
            model_type = "E5_Mistral"
            # This is a very large model, needs GPU
            model = SentenceTransformer(model_id, device=device)

        elif "scibert" in model_id:
            model_type = "AutoModel_Mean"
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModel.from_pretrained(model_id).to(device)

        elif "specter2_base" in model_id:
            model_type = "AdapterModel_CLS"
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoAdapterModel.from_pretrained(model_id).to(device)
            adapter_name = "allenai/specter2"
            model.load_adapter(adapter_name, source="hf", set_active=True)

        print(f"Loaded {model_id} in {time.time() - start_time:.2f} seconds.")

    except Exception as e:
        print(f"Failed to load {model_id}: {e}")
        continue

    # --- 1. Create Chunk Embeddings (The "Database") ---
    chunk_embeddings = None
    if model_type == "SentenceTransformer":
        chunk_embeddings = model.encode(hard_test_chunks, normalize_embeddings=True)
    elif model_type == "E5_Mistral":
        # E5 needs a "passage: " prefix for chunks
        passage_chunks = [f"passage: {chunk}" for chunk in hard_test_chunks]
        chunk_embeddings = model.encode(passage_chunks, normalize_embeddings=True)
    else: # SciBERT and Specter
        encoded_input = tokenizer(hard_test_chunks, padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)

        if model_type == "AutoModel_Mean":
            chunk_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        elif model_type == "AdapterModel_CLS":
            chunk_embeddings = cls_pooling(model_output)

        chunk_embeddings = F.normalize(chunk_embeddings, p=2, dim=1).cpu().numpy()

    chunk_embeddings = torch.tensor(chunk_embeddings).to(device)


    # --- 2. Run All Queries ---
    for q_name, q_text in test_queries:
        print(f"\n  QUERY: '{q_name}'")

        query_vector = None
        if model_type == "SentenceTransformer":
            query_vector = model.encode(q_text, normalize_embeddings=True)
        elif model_type == "E5_Mistral":
            # E5 needs a "query: " prefix for queries
            query_vector = model.encode(f"query: {q_text}", normalize_embeddings=True)
        else: # SciBERT and Specter
            encoded_input = tokenizer(q_text, padding=True, truncation=True, return_tensors='pt').to(device)
            with torch.no_grad():
                model_output = model(**encoded_input)

            if model_type == "AutoModel_Mean":
                query_vector = mean_pooling(model_output, encoded_input['attention_mask'])
            elif model_type == "AdapterModel_CLS":
                query_vector = cls_pooling(model_output)

            query_vector = F.normalize(query_vector, p=2, dim=1).cpu().numpy()

        query_vector = torch.tensor(query_vector).to(device)

        # 3. Calculate Scores
        scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)
        top_scores, top_indices = torch.topk(scores, k=3)

        # 4. Print results
        for i in range(3):
            score = top_scores[0][i].item()
            chunk_index = top_indices[0][i].item()
            print(f"    {i+1}. [Chunk {chunk_index}] (Score: {score:.4f})")

print("\n" + "="*80)
print("✅ Bake-off Complete.")


🚀 TESTING MODEL: BAAI/bge-m3
Loaded BAAI/bge-m3 in 4.12 seconds.

  QUERY: 'Conceptual F=ma'
    1. [Chunk 6] (Score: 0.6012)
    2. [Chunk 1] (Score: 0.5680)
    3. [Chunk 0] (Score: 0.5372)

  QUERY: 'Hard LaTeX (Coulomb)'
    1. [Chunk 0] (Score: 0.5666)
    2. [Chunk 1] (Score: 0.4780)
    3. [Chunk 6] (Score: 0.4477)

  QUERY: 'Hard LaTeX (Gravity)'
    1. [Chunk 1] (Score: 0.5293)
    2. [Chunk 0] (Score: 0.4944)
    3. [Chunk 6] (Score: 0.4743)

  QUERY: 'Conceptual Math'
    1. [Chunk 2] (Score: 0.4666)
    2. [Chunk 4] (Score: 0.4224)
    3. [Chunk 3] (Score: 0.4166)

  QUERY: 'Keyword Chem'
    1. [Chunk 4] (Score: 0.6618)
    2. [Chunk 5] (Score: 0.4095)
    3. [Chunk 1] (Score: 0.3315)

  QUERY: 'Keyword Chem 2'
    1. [Chunk 5] (Score: 0.7637)
    2. [Chunk 4] (Score: 0.3767)
    3. [Chunk 6] (Score: 0.3334)

🚀 TESTING MODEL: intfloat/e5-mistral-7b-instruct
Failed to load intfloat/e5-mistral-7b-instruct: cannot import name 'LossKwargs' from 'transformers.utils' (/usr/loca

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Loaded allenai/specter2_base in 0.90 seconds.

  QUERY: 'Conceptual F=ma'


RuntimeError: self must be a matrix

In [ ]:
# === 3. Load Our Winning Model ===
model_name = "BAAI/bge-m3"
print(f"Loading model: {model_name}...")
model = SentenceTransformer(model_name)
print("Model loaded.")

Loading model: BAAI/bge-m3...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Model loaded.


In [ ]:
# === 4. Create Our "Vector Database" ===
print("Generating embeddings for all 7 chunks...")
# We use normalize_embeddings=True to make cosine similarity calculation faster
chunk_embeddings = model.encode(hard_test_chunks, normalize_embeddings=True)
# Convert to a PyTorch tensor
chunk_embeddings = torch.tensor(chunk_embeddings)
print(f"Created 'database' tensor of shape: {chunk_embeddings.shape}")

Generating embeddings for all 7 chunks...
Created 'database' tensor of shape: torch.Size([7, 1024])


In [ ]:
# === 5. Define the Retrieval Stress-Test Queries ===
test_queries = [
    # 1. User's Conceptual Query
    "What is the relation between force and acceleration?",

    # 2. Hard LaTeX Query (Coulomb)
    "Find information on $\vec{F} = k_e \frac{q_1 q_2}{r^2} \hat{r}$",

    # 3. Hard LaTeX "Nuisance" Query (Gravity)
    "Find information on $\vec{F} = G \frac{m_1 m_2}{r^2} \hat{r}$",

    # 4. Math Conceptual Query
    "How do I find the area under a curve?",

    # 5. Chemistry Keyword Query
    "What is a mole?",

    # 6. Benzene Query
    "What is Benzene ($C_6H_6$)?"
]
print("\n" + "="*50)
print("🚀 RUNNING MANUAL RETRIEVAL STRESS-TEST 🚀")
print("="*50 + "\n")





🚀 RUNNING MANUAL RETRIEVAL STRESS-TEST 🚀



<>:7: SyntaxWarning: invalid escape sequence '\h'
<>:10: SyntaxWarning: invalid escape sequence '\h'
<>:7: SyntaxWarning: invalid escape sequence '\h'
<>:10: SyntaxWarning: invalid escape sequence '\h'
/tmp/ipython-input-3276775325.py:7: SyntaxWarning: invalid escape sequence '\h'
  "Find information on $\vec{F} = k_e \frac{q_1 q_2}{r^2} \hat{r}$",
/tmp/ipython-input-3276775325.py:10: SyntaxWarning: invalid escape sequence '\h'
  "Find information on $\vec{F} = G \frac{m_1 m_2}{r^2} \hat{r}$",


In [ ]:
for query in test_queries:
    print(f"\nQUERY: '{query}'")

    # 1. Create the query embedding
    query_vector = model.encode(query, normalize_embeddings=True)
    query_vector = torch.tensor(query_vector)

    # 2. Calculate Cosine Similarity
    # This is the "search"
    # We multiply the single query vector (1, 1024)
    # by the transposed chunk database (1024, 7)
    # The result is a (1, 7) tensor of scores.
    scores = torch.mm(query_vector.unsqueeze(0), chunk_embeddings.T)

    # 3. Get the top results
    # We sort the scores and get the top 3
    top_scores, top_indices = torch.topk(scores, k=3)

    # 4. Print results
    print("--- Top 3 Results ---")
    for i in range(3):
        score = top_scores[0][i].item()
        chunk_index = top_indices[0][i].item()
        retrieved_text = hard_test_chunks[chunk_index]

        print(f"  {i+1}. [Chunk {chunk_index}] (Score: {score:.4f})")
        print(f"      > {retrieved_text[:100]}...")
    print("---")


QUERY: 'What is the relation between force and acceleration?'
--- Top 3 Results ---
  1. [Chunk 6] (Score: 0.6012)
      > Newton's Second Law (Dynamics) states that the force $ec{F}$ acting on a body is equal to the mass ...
  2. [Chunk 1] (Score: 0.5680)
      > Newton's Law of Universal Gravitation states that any two bodies in the universe attract each other ...
  3. [Chunk 0] (Score: 0.5372)
      > Coulomb's Law describes the electrostatic force $ec{F}$ between two stationary point charges, $q_1$...
---

QUERY: 'Find information on $ec{F} = k_e rac{q_1 q_2}{r^2} \hat{r}$'
--- Top 3 Results ---
  1. [Chunk 0] (Score: 0.5666)
      > Coulomb's Law describes the electrostatic force $ec{F}$ between two stationary point charges, $q_1$...
  2. [Chunk 1] (Score: 0.4780)
      > Newton's Law of Universal Gravitation states that any two bodies in the universe attract each other ...
  3. [Chunk 6] (Score: 0.4477)
      > Newton's Second Law (Dynamics) states that the force $ec{F}$ a